In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 17.3 MB/s eta 0:01:10tcmalloc: large alloc 1147494400 bytes == 0x55974f9a0000 @  0x7f70aa327615 0x559716a3d17c 0x559716b1d47a 0x559716a3ff9d 0x559716b31d4d 0x559716ab3ec8 0x559716aaea2e 0x559716a4188a 0x559716ab3d30 0x559716aaea2e 0x559716a4188a 0x559716ab0719 0x559716b32b76 0x559716aafd95 0x559716b32b76 0x559716aafd95 0x559716b32b76 0x559716aafd95 0x559716a41ce9 0x559716a85579 0x559716a40902 0x559716ab3c4d 0x559716aaea2e 0x559716a4188a 0x559716ab0719 0x559716aaea2e 0x559716a4188a 0x559716aaf8f6 0x559716a417aa 0x559716aafb4f 0x559716aaea2e
     |████████████████▌               | 1055.7 MB 17.7 MB/s eta 0:00:56tcmalloc: large alloc 1434370048 bytes == 0x559793ff6000 @  0x7f70aa327615 0x559716a3d17c 0x559716b1d47a 0x559716a3ff9d 0x559716b31d4d 0x559716ab3ec8 0x559716aaea2e 0x559716a4188a 0x559716ab3d30 0x559716aaea2e 0x559716a4188a 0x559716ab0719 0x559716b32b76 0x559716aa

In [3]:
from mmcv import collect_env
collect_env()

{'CUDA available': True,
 'CUDA_HOME': '/usr/local/cuda',
 'GCC': 'x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0',
 'GPU 0': 'Tesla P100-PCIE-16GB',
 'MMCV': '1.5.0',
 'MMCV CUDA Compiler': '11.1',
 'MMCV Compiler': 'GCC 7.3',
 'NVCC': 'Cuda compilation tools, release 11.1, V11.1.105',
 'OpenCV': '4.1.2',
 'PyTorch': '1.9.0+cu111',
 'PyTorch compiling details': 'PyTorch built with:\n  - GCC 7.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.1.2 (Git Hash 98be7e8afa711dc9b66c8ff3504129cb82013cdb)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.1\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-genco

In [4]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.9.0+cu111 True
2.23.0
11.1
GCC 7.3


In [5]:
%cd ..
%cd ..

/content
/


In [6]:
!mkdir content/mmdetection/data
!mkdir content/mmdetection/data/coco

In [13]:
!cp -a content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/coco/* content/mmdetection/data/coco/

In [14]:
!cp content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/coco/config/coco_detection_base.py content/mmdetection/configs/_base_/datasets

In [15]:
!cp content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/coco/config/faster_rcnn_r50_fpn_1x_coco_base.py content/mmdetection/configs/faster_rcnn

In [16]:
!cp content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/coco/config/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco_base.py content/mmdetection/configs/faster_rcnn

In [17]:
!cp content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/coco/config/coco.py content/mmdetection/mmdet/datasets/

In [18]:
!cp content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/coco/config/faster_rcnn_r50_fpn_base.py content/mmdetection/configs/_base_/models

In [21]:
%cd content/mmdetection

/content/mmdetection


In [33]:
import mmcv
from mmcv.runner import load_checkpoint

from mmdet.apis import inference_detector, show_result_pyplot
from mmdet.models import build_detector

In [34]:
from mmcv import Config
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco_base.py')
print(cfg)

Config (path: ./configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco_base.py): {'model': {'type': 'FasterRCNN', 'backbone': {'type': 'ResNet', 'depth': 50, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': False}, 'norm_eval': True, 'style': 'caffe', 'init_cfg': {'type': 'Pretrained', 'checkpoint': 'open-mmlab://detectron2/resnet50_caffe'}}, 'neck': {'type': 'FPN', 'in_channels': [256, 512, 1024, 2048], 'out_channels': 256, 'num_outs': 5}, 'rpn_head': {'type': 'RPNHead', 'in_channels': 256, 'feat_channels': 256, 'anchor_generator': {'type': 'AnchorGenerator', 'scales': [8], 'ratios': [0.5, 1.0, 2.0], 'strides': [4, 8, 16, 32, 64]}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [1.0, 1.0, 1.0, 1.0]}, 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 1.0}}, 'roi_head': {'type': 'StandardRoI

In [35]:
print(cfg.dataset_type)
print(cfg.data_root)
print(cfg.data.test.type)
#print(cfg.data.test.data_root)
print(cfg.data.test.ann_file)
print(cfg.data.test.img_prefix)
# cfg.data.test.ann_file = 'data/coco/instances_test2017.json'
# cfg.data.test.img_prefix = 'data/coco/test/'

CocoDataset
data/coco/
CocoDataset
data/coco/instances_test2017_trunc.json
data/coco/test/


In [36]:
print(cfg.data.train.type)
#print(cfg.data.train.data_root)
print(cfg.data.train.ann_file)
print(cfg.data.train.img_prefix)
# cfg.data.train.ann_file = 'data/coco/instances_train2017_trunc.json'
# cfg.data.train.img_prefix = 'data/coco/train'
# print(cfg.data.train.ann_file)
# print(cfg.data.train.img_prefix)

CocoDataset
data/coco/instances_train2017_trunc.json
data/coco/train/


In [37]:
print(cfg.data.val.type)
#print(cfg.data.val.data_root)
print(cfg.data.val.ann_file)
print(cfg.data.val.img_prefix)
# cfg.data.val.ann_file = 'data/coco/instances_val2017_trunc.json'
# cfg.data.val.img_prefix = 'data/coco/val'
# print(cfg.data.val.ann_file)
# print(cfg.data.val.img_prefix)

CocoDataset
data/coco/instances_val2017_trunc.json
data/coco/val/


In [38]:
# modify num classes of the model in box head
print(cfg.model.roi_head.bbox_head.num_classes)
# If we need to finetune a model based on a pre-trained detector, we need to
# use load_from to set the path of checkpoints.
print(cfg.load_from)

80
None


In [39]:
# Set up working dir to save files and logs.
#print(cfg.work_dir)

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02/8
#cfg.lr_config.warmup = None
cfg.log_config.interval = 10

print(cfg.optimizer.lr)
print(cfg.lr_config.warmup)
print(cfg.log_config.interval)
# Change the evaluation metric since we use customized dataset.
print(cfg.evaluation.metric)
cfg.evaluation.metric = 'bbox'
print(cfg.evaluation.metric)
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 12
print(cfg.evaluation.interval)
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 12
print(cfg.checkpoint_config.interval)

# Set seed thus the results are more reproducible
#print(cfg.seed)
#set_random_seed(0, deterministic=False)
#print(cfg.gpu_ids)

print(cfg.log_config.hooks)

0.0025
linear
10
bbox
bbox
12
12
[{'type': 'TextLoggerHook'}]


In [40]:
from mmdet.apis import set_random_seed

# # use load_from to set the path of checkpoints.
# cfg.load_from = 'checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# We can also use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]


In [41]:
import copy
import os.path as osp

import mmcv
import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

In [42]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=23.74s)
creating index...
index created!
loading annotations into memory...
Done (t=20.71s)
creating index...


2022-04-25 00:22:19,947 - mmdet - INFO - Start running, host: root@a87ad4497d3d, work_dir: /content/mmdetection/tutorial_exps
2022-04-25 00:22:19,948 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook   

index created!


2022-04-25 00:22:25,570 - mmdet - INFO - Epoch [1][10/989]	lr: 4.746e-05, eta: 1:48:32, time: 0.549, data_time: 0.270, memory: 5316, loss_rpn_cls: 1.5381, loss_rpn_bbox: 2.0544, loss_cls: 0.7181, acc: 90.7422, loss_bbox: 0.2115, loss: 4.5222
2022-04-25 00:22:28,429 - mmdet - INFO - Epoch [1][20/989]	lr: 9.741e-05, eta: 1:22:18, time: 0.284, data_time: 0.018, memory: 5574, loss_rpn_cls: 1.1563, loss_rpn_bbox: 1.1847, loss_cls: 0.7631, acc: 97.3633, loss_bbox: 0.1812, loss: 3.2853
2022-04-25 00:22:31,345 - mmdet - INFO - Epoch [1][30/989]	lr: 1.474e-04, eta: 1:13:54, time: 0.290, data_time: 0.019, memory: 5574, loss_rpn_cls: 0.7316, loss_rpn_bbox: 0.7117, loss_cls: 0.4307, acc: 97.5488, loss_bbox: 0.1400, loss: 2.0140
2022-04-25 00:22:34,453 - mmdet - INFO - Epoch [1][40/989]	lr: 1.973e-04, eta: 1:10:37, time: 0.309, data_time: 0.021, memory: 5677, loss_rpn_cls: 0.4105, loss_rpn_bbox: 0.5520, loss_cls: 0.3739, acc: 97.2559, loss_bbox: 0.1315, loss: 1.4679
2022-04-25 00:22:37,555 - mmdet 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 500/500, 13.3 task/s, elapsed: 38s, ETA:     0s

2022-04-25 01:20:51,514 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.01s).
Accumulating evaluation results...


2022-04-25 01:20:54,569 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.004
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.002
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.008
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.008
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.008
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.004
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=0.95s).
